In [1]:
# !python -m spacy download en_core_web_sm

In [1]:
import os
os.chdir("/home/congdc/hdd/Project/generative-banner")

In [2]:
from PIL import Image
import numpy as np
import cv2
import pandas as pd
import re
from collections import Counter
import nltk
import underthesea
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
import shutil
import transformers
import spacy

from preprocessing.preprocess import clean_text, translater, gen_prompt
DEVICE = "cuda"

In [22]:
TRAIN_PATH  = "/home/congdc/hdd/Project/data/train/"
INPUT_IMAGES = "/home/congdc/hdd/Project/data/processed/images_enhanced/"
OUTPUT_IMAGES = "/home/congdc/hdd/Project/data/processed/images_train/"
OUTPUT_CAPTION = "/home/congdc/hdd/Project/data/processed/caption_train/"

In [4]:
df = pd.read_csv(f"{TRAIN_PATH}info.csv")

captions = list(df['caption'])
description = list(df['description'])
moreInfo = list(df['moreInfo'])

list_info = zip(captions, description, moreInfo)
bannerImage = list(df['bannerImage'])

# Xử lý dữ liệu

1. Làm sạch dữ liệu tiếng việt

In [5]:
list_promt_1 = []
for info in tqdm(list_info):
    cap, desc, m_info = info
    promt_clean = clean_text(cap)
    list_promt_1.append(promt_clean)

1362it [00:00, 567405.85it/s]


In [6]:
list_promt_1[:20]

['áo phủ xe 3 lớp tráng nhôm cách nhiệt, chống thấm nước, bảo vệ xe khỏi lo bẻ trộm gương.',
 'mẫu thảm lót sàn mới nhất cho các dòng xe từ 4-7 chỗ. sang trọng, bền đẹp, giảm ồn.',
 'tăng độ sang trọng, lịch lãm của xế yêu. với thảm trải sàn ô tô cao cấp 6d.',
 'bán lỗ toàn bộ thảm sàn ô tô cao cấp 6d cho xe 5-7-24 chỗ. sạch sang, chống nước, chống ồn',
 'bạt phủ xe 3 lớp, chống nắng nóng bụi bẩn, chống nước, chống xước xe, bảo vệ xe toàn diện',
 'bạt phủ xe ô tô 3 lớp-chống nắng, chống mưa, chống trộm gương, bảo vệ xe chỉ với 2 phút',
 'usb nhạc bolero, trữ tình, nhạc trẻ, remix. lên nhạc với 99% các dòng xe. usb cho ô tô.',
 'bạt phủ ô tô cách nhiệt 3 lớp-chống xước xe, chống nắng nóng bụi bẩn, chống thấm nước.',
 'chống nóng-chống cháy nổ-chống mưa bụi với bạt phủ xe 3 lớp tráng nhôm phản quang',
 'một bộ thảm da lót sàn 6d thật sự  cần thiết cần cho xe 4-7 chỗ. đa năng mà còn tiết kiệm',
 'mẫu thảm lót sàn mới cuối năm nay cho các dòng xe từ 4-7 chỗ. sang trọng, dễ vệ sinh',
 'đăng

2. Chuyển đổi ngôn ngữ

In [7]:
list_promt_2 = []
for promt in tqdm(list_promt_1):
    promt_eng = translater(promt)
    list_promt_2.append(promt_eng)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1362/1362 [03:04<00:00,  7.40it/s]


In [8]:
list_promt_2[:5]

['3 - ply foil car coating, thermal insulation, waterproof, to protect the car from the risk of stealing the mirrors.',
 'The latest floor mats for four-to seven-seater cars, luxurious, durable, noise-reducing.',
 "It's a luxury, elegant car with high-end 6D car carpeting.",
 'All-car floor mats, 6d, 5-7-24 seats, clean, water-resistant, noise-resistant.',
 '3-layer canvas, heat, dirt, water, scratch protection, full coverage']

3. Gen promt

In [9]:
list_promt_3 = []
for promt in tqdm(list_promt_2):
    promt = gen_prompt(promt)
    list_promt_3.append(promt)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1362/1362 [00:00<00:00, 78569.65it/s]


In [10]:
for i, promt in enumerate(list_promt_3[:5]):
    print(f"line {i}: {promt}")

line 0: x ply foil car coating, thermal insulation, waterproof, to protect the car from the risk of stealing the mirrors
line 1: the latest floor mats for four to seven seater cars, luxurious, durable, noise reducing
line 2: it's a luxury, elegant car with high end x car carpeting
line 3: all car floor mats, x, x x seats, clean, water resistant, noise resistant
line 4: x layer canvas, heat, dirt, water, scratch protection, full coverage


# Lưu kết quả

In [12]:
def save_sd_train(name_imgs, list_promt):
    for name_img, promt in tqdm(zip(name_imgs, list_promt)):
        promt = promt.lower()
        
        name_img = name_img.replace('.', '_out.')
        input_image = f"{INPUT_IMAGES}{name_img}"
        output_image = input_image.replace(name_img, f"{promt}.jpg").replace(INPUT_IMAGES, OUTPUT_IMAGES)
        try:
            shutil.copy(input_image, output_image)
        except:
            pass
save_sd_train(bannerImages_train, list_promt)

In [23]:
def save_pixart_train(name_imgs, list_promt):
    os.makedirs(OUTPUT_CAPTION, exist_ok = True)
    for name_img, promt in tqdm(zip(name_imgs, list_promt)):
        promt = promt.lower()
        
        name_img_enhance = name_img.replace('.', '_out.')

        # image
        input_image = f"{INPUT_IMAGES}{name_img_enhance}"
        output_image = input_image.replace(INPUT_IMAGES, OUTPUT_IMAGES)
        shutil.copy(input_image, output_image)

        # caption 
        name_caption = name_img.split(".")[0] + ".txt"
        caption_path  = f"{OUTPUT_CAPTION}{name_caption}"
        with open(file_name, "w") as file:
            file.write(content_to_write)

        
save_pixart_train(bannerImage, list_promt_3)

0it [00:00, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: '/home/congdc/hdd/Project/data/processed/images_enhanced/3_out.jpg'

test

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("NlpHUST/ner-vietnamese-electra-base")
model = AutoModelForTokenClassification.from_pretrained("NlpHUST/ner-vietnamese-electra-base")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "Liên quan vụ việc CSGT bị tố đánh dân, trúng một cháu nhỏ đang ngủ, đang lan truyền trên mạng xã hội, Đại tá Nguyễn Văn Tảo, Phó Giám đốc Công an tỉnh Tiền Giang vừa có cuộc họp cùng Chỉ huy Công an huyện Châu Thành và một số đơn vị nghiệp vụ cấp tỉnh để chỉ đạo làm rõ thông tin."

ner_results = nlp(example)
print(ner_results)

In [ ]:
# nlp = spacy.load("en_core_web_sm")